In [1]:
import pandas as pd
github_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv"
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [2]:
!pip install sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [3]:
from sentence_transformers import SentenceTransformer
df = df.iloc[:300]
model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)
answer_llm = df.iloc[0].answer_llm
embedding_vector = embedding_model.encode(answer_llm)
embedding_vector[0]

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

-0.42244658

In [8]:
from tqdm.auto import tqdm
evaluations_df = []
m = zip(df['answer_orig'].values.tolist(),df['answer_llm'].values.tolist())
for answer_orig, answer_llm in tqdm(list(m)):
  v_llm = embedding_model.encode(answer_llm)
  v_orig = embedding_model.encode(answer_orig)
  evaluations_df.append([v_llm.dot(v_orig)])
evaluations_df = pd.DataFrame(evaluations_df,columns=['score'])
evaluations_df.describe()

  0%|          | 0/300 [00:00<?, ?it/s]

,score
count,300.000000
mean,27.495996
std,6.384742
min,4.547925
25%,24.307846
50%,28.336873
75%,31.674308
max,39.476013


In [10]:
round(evaluations_df.describe().loc['75%','score'],2)

31.67

In [11]:
import numpy as np
def normalize_vector(v):
  norm = np.sqrt((v * v).sum())
  v_norm = v / norm
  return v_norm

In [12]:
evaluations_cosine_score_df = []
m = zip(df['answer_orig'].values.tolist(),df['answer_llm'].values.tolist())
for answer_orig, answer_llm in tqdm(list(m)):
  v_llm_norm = normalize_vector(embedding_model.encode(answer_llm))
  v_orig_norm = normalize_vector(embedding_model.encode(answer_orig))
  evaluations_cosine_score_df.append([v_llm_norm.dot(v_orig_norm)])
evaluations_cosine_score_df = pd.DataFrame(evaluations_cosine_score_df,columns=['cosine_score'])
evaluations_cosine_score_df.describe()

  0%|          | 0/300 [00:00<?, ?it/s]

,cosine_score
count,300.000000
mean,0.728393
std,0.157755
min,0.125357
25%,0.651273
50%,0.763761
75%,0.836235
max,0.958796


In [14]:
round(evaluations_cosine_score_df.describe().loc['75%','cosine_score'],2)

0.84

In [15]:
!pip install rouge

In [21]:
from rouge import Rouge
rouge_scorer = Rouge()
r = df.loc[10,:]
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [22]:
rouge_1 = scores['rouge-1']['f']
rouge_2 = scores['rouge-2']['f']
rouge_l = scores['rouge-l']['f']
rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3
rouge_avg

0.35490034990035496

In [27]:
scores_df = []
m = zip(df['answer_orig'].values.tolist(),df['answer_llm'].values.tolist())
for answer_orig, answer_llm in tqdm(list(m)):
  scores = rouge_scorer.get_scores(answer_orig, answer_llm)[0]
  rouge_1 = scores['rouge-1']['f']
  rouge_2 = scores['rouge-2']['f']
  rouge_l = scores['rouge-l']['f']
  rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3
  scores_df.append([rouge_1,rouge_2,rouge_l,rouge_avg])
scores_df = pd.DataFrame(scores_df,columns=['rouge_1','rouge_2','rouge_l','rouge_avg'])
scores_df['rouge_2'].mean()

  0%|          | 0/300 [00:00<?, ?it/s]

0.20696501983423318